In [22]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import argparse 

import os
import tarfile

In [5]:
train=pd.read_csv('train-scene/train.csv')
test=pd.read_csv('test.csv')
image_path='train-scene/train/'

In [6]:
from scipy.misc import imresize
train_img=[]
for i in range(len(train)):
    temp_img=image.load_img(image_path+train['image_name'][i],target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    train_img.append(temp_img)

In [7]:
train_img=np.array(train_img)
train_img=preprocess_input(train_img)

In [8]:
test_img=[]
for i in range(len(test)):
    temp_img=image.load_img(image_path+test['image_name'][i],target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    test_img.append(temp_img)

In [9]:
test_img=np.array(test_img)
test_img=preprocess_input(test_img)

In [10]:
model = ResNet50(weights='imagenet', include_top=False)

C:\Users\pf416e\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 84s 1us/step


In [11]:
features_train=model.predict(train_img)
features_test=model.predict(test_img)

In [13]:
train_x=features_train.reshape(17034,100352)
train_y=np.asarray(train['label'])

In [14]:
train_y=np.asarray(train['label'])
# performing one-hot encoding for the target variable

train_y=pd.get_dummies(train_y)
train_y=np.array(train_y)
# creating training and validation set

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)


In [15]:
# creating a mlp model
from keras.layers import Dense, Activation
model=Sequential()

model.add(Dense(1000, input_dim=100352, activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)

model.add(Dense(500,input_dim=1000,activation='sigmoid'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)

model.add(Dense(150,input_dim=500,activation='sigmoid'))
keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)

model.add(Dense(units=6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# fitting the model 

model.fit(X_train, Y_train, epochs=20, batch_size=30,validation_data=(X_valid,Y_valid))


Train on 11923 samples, validate on 5111 samples
Epoch 1/20
11923/11923 [==============================] - 710s 60ms/step - loss: 0.3025 - acc: 0.9002 - val_loss: 0.2386 - val_acc: 0.9077
Epoch 2/20
11923/11923 [==============================] - 720s 60ms/step - loss: 0.1747 - acc: 0.9417 - val_loss: 0.2260 - val_acc: 0.9184
Epoch 3/20
11923/11923 [==============================] - 743s 62ms/step - loss: 0.1572 - acc: 0.9472 - val_loss: 0.2174 - val_acc: 0.9213
Epoch 4/20
11923/11923 [==============================] - 735s 62ms/step - loss: 0.1327 - acc: 0.9567 - val_loss: 0.2110 - val_acc: 0.9300
Epoch 5/20
11923/11923 [==============================] - 701s 59ms/step - loss: 0.1167 - acc: 0.9619 - val_loss: 0.2253 - val_acc: 0.9212
Epoch 6/20
11923/11923 [==============================] - 512s 43ms/step - loss: 0.1205 - acc: 0.9597 - val_loss: 0.2472 - val_acc: 0.9145
Epoch 7/20
11923/11923 [==============================] - 513s 43ms/step - loss: 0.1192 - acc: 0.9614 - val_loss: 0.2

In [16]:
validation_x=features_test.reshape(7301,100352)

In [17]:
classes = model.predict(validation_x, batch_size=30)
class_labels = np.argmax(classes, axis=1)
class_labels_dt=pd.DataFrame(class_labels)
class_labels_dt.columns=['label']

In [18]:
frames=[test,class_labels_dt]
result = pd.concat(frames,axis=1)

In [19]:
result.to_csv('intel_model_resnet50_20_30.csv')

In [23]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_intel_trained_resnet50_model_20_30.h5'

In [24]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\pf416e\Desktop\AV\intel\saved_models\keras_intel_trained_resnet50_model_20_30.h5 
